In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#modeling
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
#load dataset
df = pd.read_csv(r'data/stud.csv')

In [3]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


### Prepare X and y variables

In [4]:
X = df.drop(columns=['math score'], axis=1)
y = df['math score']

### Create Column Transformer with 3 Types of Transformers 

In [5]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [6]:
X = preprocessor.fit_transform(X)

In [8]:
X.shape

(1000, 19)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

### Evaluation Function

In [11]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)
    return mae, mse, rmse, r2

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Support Vector Regressor": SVR(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor(),
    "CatBoost Regressor": CatBoostRegressor(verbose=0),
    "XGBoost Regressor": XGBRegressor(),
    
}
model_list = []
r2_list = []
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    train_mae, train_mse, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    test_mae, test_mse, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)
   
    print(f"{list(models.keys())[i]}: MAE={test_mae}, MSE={test_mse}, RMSE={test_rmse}, R2={test_r2}")
    print(list(models.keys())[i])
    
    model_list.append(list(models.keys())[i])
    print("Model Performance for Training set:")
    print("Root mean squared error: {:.4f}".format(train_rmse))
    print("R2 score: {:.4f}".format(train_r2))
    
    print("Model Performance for Test set:")
    print("Root mean squared error: {:.4f}".format(test_rmse))
    print("R2 score: {:.4f}".format(test_r2))
    print("-----------------------------------------------------")
    r2_list.append(test_r2*100)

Linear Regression: MAE=4.21476314247485, MSE=29.095169866715487, RMSE=5.393993869732843, R2=0.8804332983749565
Linear Regression
Model Performance for Training set:
Root mean squared error: 5.3231
R2 score: 0.8743
Model Performance for Test set:
Root mean squared error: 5.3940
R2 score: 0.8804
-----------------------------------------------------
Ridge Regression: MAE=4.211100688014261, MSE=29.056272192348324, RMSE=5.390387016935642, R2=0.8805931485028737
Ridge Regression
Model Performance for Training set:
Root mean squared error: 5.3233
R2 score: 0.8743
Model Performance for Test set:
Root mean squared error: 5.3904
R2 score: 0.8806
-----------------------------------------------------
Lasso Regression: MAE=5.157881810347764, MSE=42.50641683841164, RMSE=6.519694535667422, R2=0.8253197323627852
Lasso Regression
Model Performance for Training set:
Root mean squared error: 6.5938
R2 score: 0.8071
Model Performance for Test set:
Root mean squared error: 6.5197
R2 score: 0.8253
----------

: 

In [ ]:
pd.DataFrame(list(model_list, r2_list), columns=['Model', 'R2 Score']).sort_values(by=['R2_Score'], ascending=False)